<a href="https://colab.research.google.com/github/jeffheaton/present/blob/master/youtube/gpu/keras-dual-gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jeff Heaton

You can follow me at any of:

* [YouTube](https://www.youtube.com/user/HeatonResearch)
* [Website](https://www.heatonresearch.com/)
* [GitHub](https://github.com/jeffheaton)
* [Twitter](https://twitter.com/jeffheaton)


## Multi-GPU Support

Keras makes it easy to use more than one GPU for neural network training or scoring. This tutorial shows how to train a model for the [Cats vs Dogs](https://www.kaggle.com/c/dogs-vs-cats) dataset. Not all models will necessarily benefit from multiple GPUs.  Generally larger batch sizes and more complex neural networks benefit from multiple GPUs.

The technique presented in this notebook can train with between 1 and 8 GPUs on a single host.  It is also possable to train larger numbers of GPUs on multiple hosts; however, a slightly different approach is needed. 

First, we will list what GPUs are available on the system.

In [1]:
from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()

def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for d in devices:
    t = d.device_type
    name = d.physical_device_desc
    l = [item.split(':',1) for item in name.split(", ")]
    name_attr = dict([x for x in l if len(x)==2])
    dev = name_attr.get('name', 'Unnamed device')
    print(f" {d.name} || {dev} || {t} || {sizeof_fmt(d.memory_limit)}")

 /device:CPU:0 || Unnamed device || CPU || 256.0 MiB
 /device:GPU:0 ||  RTX A6000 || GPU || 44.0 GiB


# Obtain Cats vs Dogs Dataset

First, we obain the Cats vs Dogs dataset.  We use the [tensorflow_datasets](https://www.tensorflow.org/datasets) library access this data. Any data can be used, **tensorflow_datasets** makes loading common datasets a simple process.

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

BATCH_SIZE = 32
GPUS = ["GPU:0", "GPU:1"]

def process(image, label):
    image = tf.image.resize(image, [299, 299]) / 255.0
    return image, label

strategy = tf.distribute.MirroredStrategy( GPUS )
print('Number of devices: %d' % strategy.num_replicas_in_sync) 

batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

dataset = tfds.load("cats_vs_dogs", split=tfds.Split.TRAIN, as_supervised=True)
dataset = dataset.map(process).shuffle(500).batch(batch_size)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling cats_vs_dogs-train.tfrecord...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Dataset cats_vs_dogs downloaded and prepared to /home/jeff/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


# Setup Distributed Training

Training with multiple GPUs is not much different than training with a single GPU.  Wrapping the model creation and compilation with a mirror strategy scope is all that is required.

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import time

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

EPOCHS = 5
LR = 0.001 

tf.get_logger().setLevel('ERROR')

start = time.time()
with strategy.scope():
    model = tf.keras.applications.InceptionResNetV2(weights=None, classes=2)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=[tf.keras.metrics.sparse_categorical_accuracy]
    )

model.fit(dataset, epochs=EPOCHS)

elapsed = time.time()-start
print (f'Training time: {hms_string(elapsed)}')

Epoch 1/5
364/364 [==============================] - 272s 573ms/step - loss: 0.6940 - sparse_categorical_accuracy: 0.6598
Epoch 2/5
364/364 [==============================] - 209s 573ms/step - loss: 0.4446 - sparse_categorical_accuracy: 0.7963
Epoch 3/5
364/364 [==============================] - 209s 572ms/step - loss: 0.2588 - sparse_categorical_accuracy: 0.8929
Epoch 4/5
364/364 [==============================] - 208s 571ms/step - loss: 0.1797 - sparse_categorical_accuracy: 0.9261
Epoch 5/5
364/364 [==============================] - 209s 572ms/step - loss: 0.1386 - sparse_categorical_accuracy: 0.9453
Training time: 0:18:30.76


* 15:54.81 - Dual Quadro RTX 8000
* 18:30.76 - Single RTX A6000
* 24:17.07 - Single TITAN RTX
* 24:48.10 - Single Tesla V100-SXM2-16GB
* 26:23.19 - Single Quadro RTX 8000
* 37:48.45 - Single Tesla P100-PCIE-16GB
* 50:36.50 - Single Quadro RTX 5000
* 1:10:08.54 - Single Tesla T4

Epoch 1/5
364/364 [==============================] - 453s 1s/step - loss: 0.5174 - sparse_categorical_accuracy: 0.7493
Epoch 2/5
364/364 [==============================] - 301s 826ms/step - loss: 0.2556 - sparse_categorical_accuracy: 0.8929
Epoch 3/5
364/364 [==============================] - 309s 848ms/step - loss: 0.1792 - sparse_categorical_accuracy: 0.9267
Epoch 4/5
364/364 [==============================] - 306s 842ms/step - loss: 0.1344 - sparse_categorical_accuracy: 0.9452
Epoch 5/5
364/364 [==============================] - 307s 844ms/step - loss: 0.1104 - sparse_categorical_accuracy: 0.9571
Training time: 0:30:46.31

8000
Epoch 1/5
364/364 [==============================] - 178s 489ms/step - sparse_categorical_accuracy: 0.7405 - loss: 0.5237
Epoch 2/5
364/364 [==============================] - 180s 494ms/step - sparse_categorical_accuracy: 0.8725 - loss: 0.2928
Epoch 3/5
364/364 [==============================] - 179s 492ms/step - sparse_categorical_accuracy: 0.9205 - loss: 0.1900
Epoch 4/5
364/364 [==============================] - 179s 491ms/step - sparse_categorical_accuracy: 0.9402 - loss: 0.1445
Epoch 5/5
364/364 [==============================] - 178s 490ms/step - sparse_categorical_accuracy: 0.9536 - loss: 0.1136
Training time: 0:16:14.80

Epoch 1/5
727/727 [==============================] - 310s 427ms/step - loss: 0.5647 - sparse_categorical_accuracy: 0.7119
Epoch 2/5
727/727 [==============================] - 308s 424ms/step - loss: 0.3291 - sparse_categorical_accuracy: 0.8558
Epoch 3/5
727/727 [==============================] - 307s 422ms/step - loss: 0.2029 - sparse_categorical_accuracy: 0.9166
Epoch 4/5
727/727 [==============================] - 306s 421ms/step - loss: 0.1567 - sparse_categorical_accuracy: 0.9347
Epoch 5/5
727/727 [==============================] - 306s 421ms/step - loss: 0.1300 - sparse_categorical_accuracy: 0.9469
Training time: 0:26:23.19